# Annotated example of a Bering Sea optional-bio run

This example annotates one of my own real run scripts, and is targeted at members of my own research group running the Alaska region multi-BGC version of ROMS that I am working with.  It may also be useful to others to see a more complex run setup.

This script is designed to run hindcast simulations for any of 4 different Bering Sea model variants: physics-only, or with the BEST_NPZ, BIO_BANAS, or BIO_COBALT biogeochemical models.  It is also designed to run on the mox checkpoint queue if needed; this means that if the script is cancelled externally by the job scheduler, it can resume simulation without any extra modifications.

Note that there are a few hard-coded paths below that will need to be adjusted to reflect your own local setup.  These are marked with "TODO: Change for your setup!"

## Setup

We begin with some setup details before we get into the romscom-specific stuff.

As in most python scripts, we begin with some package imports:

In [1]:
import os
import sys
import argparse
from datetime import datetime, timedelta

import romscom.romscom as rc

This script was written to accept a few command-line inputs, allowing it to be tested in "dry run" mode (where it sets up all the input but doesn't actually call the ROMS executble) and to switch which biological model variant is being run.  For this example, we'll hard-wire the physics-only model with dry run on.

In [2]:
sys.argv = ['', '--dryrun', '--bio', 'phys'] # <- This line added for notebook only

# Parse input

ap = argparse.ArgumentParser()
ap.add_argument("--bio", nargs="+", choices=["bestnpz", "cobalt", "banas", "phys"], default=["bestnpz", "cobalt", "banas", "phys"])
ap.add_argument("-d", "--dryrun", action="store_true")
args = ap.parse_args()

Next, we add some setup variables related to the various configurations we may run, including the ROMS CPP flag associated with each option, the executable to be used for each, and our local mpirun command.

In [3]:
biocpp   = {"bestnpz": "BEST_NPZ", 
            "cobalt":  "BIO_COBALT", 
            "banas":   "BIO_BANAS", 
            "phys":    ""}
          
romsexec = {"bestnpz": "./romsM_bestnpz_202303150939",
            "cobalt":   "./romsM_cobalt_202303150949",
            "banas":     "./romsM_banas_202303151001",
            "phys":       "./romsM_phys_202303150930"}

mpiexec = "mpirun"


The simulations will start on Jan 15, 1990 (the first date in the boundary condition files) and run through Jan 1, 2020.


In [4]:
maxdate = datetime(2020,1,1)
dstart = datetime(1990,1,15,0,0)

## Setting up input parameters

This next section does most of the input parameter setup.  We start by importing all the default values from our ROMS Application Data folder.  The values are saved in dictionaries that now act as stand-ins for their .in file counterparts.

In [5]:
# Defaults

appfol = "/Users/kakearney/Documents/Research/Working/ReposCode/roms/bering-Apps/Apps/Bering_BGC_variants" # <- TODO: Change for your setup!

ocean   = rc.readparamfile(os.path.join(appfol, "bering_ocean.yaml"), tconvert=True)
station = rc.readparamfile(os.path.join(appfol, "bering_spos.yaml"))
ice     = rc.readparamfile(os.path.join(appfol, "bering_ipar.yaml"))

bio = {};
for btmp in args.bio:
    print(btmp)
    if btmp != "phys":
        bio[btmp] = rc.readparamfile(os.path.join(appfol, f"bering_bpar_{btmp}.yaml"))

phys


With the defaults in place, we can then begin making adjustments that are specific to this particular experiment.  In this script, I turned on some comparable diagnostics for the bio models.

In [6]:
# Turn on appropriate diagnostics:

if "banas" in args.bio:
    for k in bio["banas"]['Dout']: 
        bio["banas"]['Dout'][k] = True # BIO_BANAS: turn on all diagnostics
        
if "bestnpz" in args.bio:
    bio["bestnpz"]["Dout"]["iprod_PhS"]        = True
    bio["bestnpz"]["Dout"]["iprod_PhL"]        = True
    bio["bestnpz"]["Dout"]["iprod_MZL"]        = True
    bio["bestnpz"]["Dout"]["iprod_Cop"]        = True
    bio["bestnpz"]["Dout"]["iprod_NCaS"]       = True
    bio["bestnpz"]["Dout"]["iprod_EupS"]       = True
    bio["bestnpz"]["Dout"]["iprod_NCaO"]       = True
    bio["bestnpz"]["Dout"]["iprod_EupO"]       = True
    bio["bestnpz"]["Dout"]["iprod_Jel"]        = True
    bio["bestnpz"]["Dout"]["iprod_Ben"]        = True
    bio["bestnpz"]["Dout"]["iprod_IcePhL"]     = True
    bio["bestnpz"]["Dout"]["ipar"]             = True
    bio["bestnpz"]["Dout"]["inolims"]          = True
    bio["bestnpz"]["Dout"]["inoliml"]          = True
    bio["bestnpz"]["Dout"]["inhlims"]          = True
    bio["bestnpz"]["Dout"]["inhliml"]          = True

if "cobalt" in args.bio:
    bio["cobalt"]["Dout"]["inpp_sm"]   = True
    bio["cobalt"]["Dout"]["inpp_md"]   = True
    bio["cobalt"]["Dout"]["inpp_lg"]   = True
    bio["cobalt"]["Dout"]["inpp_di"]   = True
    bio["cobalt"]["Dout"]["ifratio"]   = True
    bio["cobalt"]["Dout"]["iprod_smz"] = True
    bio["cobalt"]["Dout"]["iprod_mdz"] = True
    bio["cobalt"]["Dout"]["iprod_lgz"] = True
    bio["cobalt"]["Dout"]["iironsed_flx"] = True
    bio["cobalt"]["Dout"]["indet_btf"]    = True
    bio["cobalt"]["Dout"]["io2_btf"]      = True

Then I altered many of the archiving time step variables.  I can just set those based on time intervals, and don't need to worry about converting to number of ROMS time steps.

In [7]:
# Time variables 

ocean['DSTART'] = dstart

# Set archiving time steps and file size

ocean['NRST'] = timedelta(days=4)
ocean['NSTA'] = timedelta(days=1)
ocean['NAVG'] = timedelta(days=4)
ocean['NHIS'] = timedelta(days=4)
ocean['NDIA'] = timedelta(days=4)
ocean['NDEFAVG'] = timedelta(days=40)
ocean['NDEFHIS'] = timedelta(days=40)
ocean['NDEFDIA'] = timedelta(days=40)

Most of the default input file paths are just placeholders, so we'll replace those with the real file paths.  We start with a few that are shared across all the bio variants:

In [8]:
# Change a few input files to reflect data folder location

datafol = "/Users/kakearney/Documents/Research/Working/mox_bumblereem/ROMS_Datasets" # <- TODO: Change for your setup!

ocean["GRDNAME"]  = f"{datafol}/grids/AlaskaGrids_Bering10K.nc"
ocean["TIDENAME"] = f"{datafol}/OTPS/tides_OTPS_Bering10K.nc"
ocean["NUDNAME"]  = f"{datafol}/initial/nudgingcoeff_Bering10K.nc"

## Running with runtodate()

This next section preps the simulation to be run using the romscom runtodate() function, run in 10-year blocks.  The 10-year thing is mainly a workaround for a ROMS input file limitation that I discovered the hard way.  

The runtodate() provides a wrapper to set up a ROMS simulation and run through the desired date, allowing for robust restarts when necessary. It organizes ROMS I/O under a 3-folder system under the user-specified simdir folder. Before calling the ROMS executable, it looks for an appropriately-named restart file under the Out subfolder. If found, it uses this restart file to initialize a run with NRREC=-1; otherwise, it will use the user-provided ININAME and NRREC values. It also adjusts the NTIMES field to reach the requested end date.
               
This procedure allows a simulation to be restarted using the same command regardless of whether it has been partially completed or not; this can be useful when running simulations on computer clusters where jobs may be cancelled and resubmitted for various queue management reasons, or to extend existing simulations with new forcing.

This function also provides the option to work through ROMS blowups. These occur when physical conditions lead to numeric instabilities. Blowups can sometimes be mitigated by reducing the model time step. When the runpastblowup option is True and runtodate encounters a blowup, it will adjust the DT parameter to the user-provided slow time step, restart the simulation from the last history file, and run for 30 days.  It will then return to the original time step and resume. Note that this time step reduction will only be attempted once; if the model still blows up, the simulation will exit and the user will need to troubleshoot the situation.

Each time the model is restarted, output file counters are incremented as specified by the addcounter option.  This preserves output that would otherwise be overwritten on restart with the same simulation name.  By default, the counter is only added to file types that modern ROMS does not check for on restart.

In [9]:
# Adjust input based on years to run 
# (template file only holds a few years worth)
# Note: Ideally we'd just run the whole thing at once.  But ROMS does *not* like
# it when you supply more than ~100 forcing multi-files (there's nothing in the
# documentation about this... but somewhere under the hood a variable must be
# allocated to only hold a limited number of characters, and if you surpass that
# things go sideways in a messy-crash-with-very-unhelpful-errors sort of way!
# So, to accomodate, we run in blocks).

blocksz = 10 # years per call, to keep file numbers under the ROMS limit

for bioname in args.bio:
    
    print(f"Preparing {bioname} simulation...")
    
    # Start with time-invariant inputs
    
    pflag = bioname == "phys"

    simdir = f"bgcmip_{bioname}"
    simname = f"bgcmip_{bioname}"

    # Set up sim folders

    fol = rc.simfolders(simdir, create=True)

    # Write accessory files

    bpar = os.path.join(fol['in'], f"{simname}_bpar.in") # bio
    ipar = os.path.join(fol['in'], f"{simname}_ipar.in") # ice
    spos = os.path.join(fol['in'], f"{simname}_spos.in") # stations

    if not pflag:
        rc.dict2standardin(bio[bioname], compress=False, file=bpar)
    rc.dict2standardin(ice, compress=False, file=ipar)
    rc.dict2standardin(station, compress=False, file=spos)

    if pflag:
        ocean['BPARNAM'] = "placeholder.in"
    else:
        ocean['BPARNAM'] = bpar
        
    ocean['IPARNAM'] = ipar
    ocean['SPOSNAM'] = spos

    # Point to correct initialization and varinfo files

    if pflag:
        ocean['ININAME'] = os.path.join(datafol, 'initial', 'ini_hindcast_unnested_Bering10K_BEST_NPZ.nc')
        ocean["CLMNAME"] = os.path.join(datafol, 'initial', 'ini_hindcast_unnested_Bering10K_BEST_NPZ.nc')
    else:
        ocean['ININAME'] = os.path.join(datafol, 'initial', f"ini_hindcast_unnested_Bering10K_{biocpp[bioname]}.nc")
        ocean["CLMNAME"] = os.path.join(datafol, 'initial', f"ini_hindcast_unnested_Bering10K_{biocpp[bioname]}.nc")

    if pflag:
        ocean['VARNAME'] = os.path.join(appfol, f"varinfo_{bioname}.dat")
    else:
        ocean['VARNAME'] = os.path.join(appfol, f"varinfo_{bioname}_scaledbry.dat")
    
    # Break simulation into blocks to avoid the (undocumented) limit on ROMS 
    # input multi-file number.
    
    for yr1 in range(dstart.year,maxdate.year,blocksz):
        
        endblock = min(maxdate, datetime(yr1+blocksz,1,1))
        yrs = range(max(yr1-1,dstart.year), min(endblock.year+2, maxdate.year+1)) # Adds a little overlap for restarting from one block to the next
        
        ocean["FRCNAME"] = [f"{datafol}/BarrowCO2/atmo_co2_barrow_1970_2020.nc", 
                            f"{datafol}/Iron/ESM4_Bering10K_iron_dust_clim.nc",
                            f"{datafol}/salinity/sss.clim.nc",
                            [f"{datafol}/CFS/{x}/CFS-atmos-northPacific-Pair-{x}.nc" for x in yrs],
                            [f"{datafol}/CFS/{x}/CFS-atmos-northPacific-Qair-{x}.nc" for x in yrs],
                            [f"{datafol}/CFS/{x}/CFS-atmos-northPacific-Tair-{x}.nc" for x in yrs],
                            [f"{datafol}/CFS/{x}/CFS-atmos-northPacific-Uwind-{x}.nc" for x in yrs],
                            [f"{datafol}/CFS/{x}/CFS-atmos-northPacific-Vwind-{x}.nc" for x in yrs],
                            [f"{datafol}/CFS/{x}/CFS-atmos-northPacific-rain-{x}.nc" for x in yrs],
                            [f"{datafol}/CFS/{x}/CFS-atmos-northPacific-swrad-{x}.nc" for x in yrs],
                            [f"{datafol}/CFS/{x}/CFS-atmos-northPacific-lwrad-{x}.nc" for x in yrs],
                            [f"{datafol}/GloFAS/GloFAS_runoff_Bering10K_{x}.nc" for x in yrs],
                            [f"{datafol}/GloFAS/GloFAS-based_nutrientflux_Bering10K_{x}.nc" for x in yrs]
                           ]
        ocean["NFFILES"] = len(ocean["FRCNAME"])

        ocean["BRYNAME"] = [f"{datafol}/WOA2018/WOA2018_Bering10K_N30_brybgc.nc",
                            [f"{datafol}/CFS/{x}/CFS-ocean-Bering10K-N30-bryocn-{x}.nc" for x in yrs],
                            [f"{datafol}/CFS/{x}/CFS-ocean-ESPER-Bering10K-N30-brycarbon-{x}.nc"  for x in yrs]
                           ]
        ocean["NBCFILES"] = len(ocean["BRYNAME"])

        # ocean['NTIMES'] = min(maxdate, datetime(yr1+blocksz,1,1)) - ocean['DSTART']

        # Reset restart/initialize marker, in case changed by one of the other bio 
        # sims (runtodate will adjust as necessary)
    
        ocean['NRREC'] = 0
    
        # Run
    
        romscmd = [mpiexec, romsexec[bioname]]
    
        status = rc.runtodate(ocean, simdir, simname, enddate=endblock, romscmd=romscmd, 
                     dryrunflag=args.dryrun)
                     
        if status != "success":
            break

Preparing phys simulation...
Running ROMS simulation
  Counter block:   1
  Start date:      1990-01-15 00:00:00
  End date:        2000-01-01 00:00:00
  ROMS command:    mpirun ./romsM_phys_202303150930
  Standard input:  bgcmip_phys/In/bgcmip_phys_01_ocean.in
  Standard output: bgcmip_phys/Log/bgcmip_phys_01_log.txt
  Standard error:  bgcmip_phys/Log/bgcmip_phys_01_err.txt
Dry run


In dry run mode, the script creates the requested input files, then reports details of the simulation it is ready to run.  If we were to run this script with the dry run off, it would proceed to actually call ROMS, complete the first 10-year block, then move on to the next one, with output added to the Out folder and standard error and output going to the Log folder.  Once the simulation reaches its end, you will have the full set of input files, log files, and output files available for your records in the specified folder.